In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

VER = 161
PART = 2

In [2]:
import pandas as pd, cudf, numpy as np, gc
pd.set_option('display.max_columns', 500)
import matplotlib.pyplot as plt

In [3]:
users = pd.read_parquet(f'../../data/infer_with_features/train_v{VER}_LB_{PART}.pqt')

In [4]:
print( users.shape )
users.head()

(41795000, 100)


,user,item,pos,wgt,score_1,score_3,score_4,score_5,score_6,score_7,score_10,score_11,score_12,score_13,score_30,score_14,score_15,score_16,score_17,score_18,score_19,score_20,score_21,score_22,score_23,score_24,score_25,score_26,buy_ratio_5,count_item_5,count_user_5,repeat_5,buy_ratio_std_5,item_day_std_5,user_day_std_5,buy_ratio_0_6,buy_ratio_1_6,buy_ratio_2_6,buy_ratio_3_6,buy_ratio_4_6,buy_ratio_5_6,buy_ratio_6_6,count_item_0_6,count_item_1_6,count_item_2_6,count_item_3_6,count_item_4_6,count_item_5_6,count_item_6_6,count_user_0_6,count_user_1_6,count_user_2_6,count_user_3_6,count_user_4_6,count_user_5_6,count_user_6_6,item_day_std_6,user_day_std_6,buy_ratio,count_item,count_user,hour_mean,day_mean,repeat,buy_ratio_std,hour_std,day_std,buy_ratio3,count_item3,count_user3,hour_mean3,day_mean3,repeat3,buy_ratio_std3,hour_std3,day_std3,prev3,next3,orders3,carts3,order_repeat3,cart_repeat3,buy_ratio2,count_item2,unique_item2,hour_mean2,day_mean2,repeat2,buy_ratio_std2,hour_std2,day_std2,ts_max7,ts_min7,ts_diff7,z1,z2,z3,z4,z5,z6
0,12899779,59625,0,-1.000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.414214,-1.0,0.071773,-1.0,-1.000,1.414214,1.414214,1.414214,1.414214,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.000000,4.0,4.0,1.000000,0.000000,NaN,NaN,0.000000,-1.00,-1.000000,-1.0,-1.0,-1.000000,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.377964,0.377964,0.000000,7.0,6.0,58189.570312,292703.84375,1.166667,0.000000,22522.156250,177373.40625,0.000000,1.0,1.0,79200.000000,338400.00000,1.000000,-1.000000,-1.000000,-1.000000,0.0,0.0,0.0,0.0,-1.0,-1.0,0.0,1.0,1.0,79200.0,338400.0,1.0,-1.0,-1.0,-1.0,0,0,0,1.000000,-1.0,-1.0,-1.0,-1.0,-1.0
1,12899779,737445,1,10.495,1.0,-3.0,-3.0,-3.0,2.0,-3.0,0.850000,3.0,1.500000,8.0,8.000,-3.000000,-3.000000,-3.000000,0.850000,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,1.0,2.0,1.0,0.022444,401.0,258.0,1.554264,0.148307,NaN,NaN,0.000000,0.00,0.166667,0.0,0.0,0.000000,0.0,4.0,7.0,6.0,3.0,3.0,3.0,9.0,4.0,4.0,4.0,3.0,3.0,2.0,7.0,2.380476,2.380476,0.019337,724.0,450.0,50406.179688,300655.90625,1.608889,0.137802,19708.617188,166473.00000,0.028571,35.0,25.0,50927.429688,349624.56250,1.400000,0.169031,21019.531250,161340.265625,15.0,12.0,0.0,1.0,-1.0,1.0,0.0,1.0,1.0,79200.0,338400.0,1.0,-1.0,-1.0,-1.0,0,0,0,0.954863,-1.0,-1.0,-1.0,-1.0,-1.0
2,12899779,1253524,2,6.000,1.0,-3.0,-3.0,-3.0,0.5,-3.0,0.050000,1.5,1.500000,7.0,6.200,-3.000000,-3.000000,-3.000000,0.050000,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,1.0,0.5,2.0,0.008696,115.0,90.0,1.277778,0.093250,NaN,NaN,-1.000000,0.00,0.000000,0.0,-1.0,-1.000000,-1.0,-1.0,1.0,2.0,1.0,-1.0,-1.0,-1.0,-1.0,1.0,2.0,1.0,-1.0,-1.0,-1.0,0.786796,0.786796,0.027273,220.0,160.0,54470.640625,285001.56250,1.375000,0.163248,19156.367188,170871.09375,0.000000,4.0,4.0,53515.000000,420715.00000,1.000000,0.000000,11709.117188,255545.156250,2.0,3.0,0.0,0.0,-1.0,-1.0,0.0,1.0,1.0,79200.0,338400.0,1.0,-1.0,-1.0,-1.0,0,0,0,0.954612,-1.0,-1.0,-1.0,-1.0,-1.0
3,12899779,731692,3,5.990,1.0,-3.0,-3.0,-3.0,0.5,-3.0,-3.000000,1.5,0.300000,4.0,10.675,-3.000000,-3.000000,-3.000000,-3.000000,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,1.0,0.5,1.0,0.140000,50.0,28.0,1.785714,0.404566,NaN,NaN,-1.000000,0.25,0.500000,-1.0,-1.0,-1.000000,0.0,-1.0,4.0,2.0,-1.0,-1.0,-1.0,1.0,-1.0,2.0,1.0,-1.0,-1.0,-1.0,1.0,1.527525,1.527525,0.162791,86.0,41.0,52135.464844,295261.03125,2.097561,0.430061,17917.339844,174344.12500,0.285714,7.0,4.0,53310.429688,485310.43750,1.750000,0.487950,11891.376953,79692.445312,3.0,4.0,0.0,2.0,-1.0,1.0,0.0,1.0,1.0,79200.0,338400.0,1.0,-1.0,-1.0,-1.0,0,0,0,0.956417,-1.0,-1.0,-1.0,-1.0,-1.0
4,12899779,1246235,4,5.935,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,0.050000,-3.0,0.900000,3.0,4.250,-3.000000,-3.000000,-3.000000,0.050000,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,1.0,0.038974,975.0,664.0,1.468374,0.208935,NaN,NaN,0.235294,0.00,0.000000,0.1,0.0,0.083333,0.0,17.0,8.0,10.0,10.0,4.0,12.0,5.0,12.0,8.0,10.0,4.0,4.0,10.0,4.0,4.391551,4.391551,0.053419,1872.0,1228.0,52150.824219,312135.43750,1.524430,0.240987,18081.47851

# Add New Items

In [5]:
# FEAT 7
df = pd.read_parquet('../../data/item_user_features/item20_LB.pqt')
df.head()

,buy_ratio7,count_item7,count_user7,hour_mean7,day_mean7,repeat7,buy_ratio_std7,hour_std7,day_std7,prev7,next7,orders7,carts7,order_repeat7,cart_repeat7
aid,,,,,,,,,,,,,,,
0,0.000000,11,10,43598.363281,310652.90625,1.100000,0.000000,22148.582031,144431.906250,9,10,0,0,-1.0,-1.0
1,0.000000,2,1,36721.000000,36721.00000,2.000000,0.000000,63.639610,63.639610,2,1,0,0,-1.0,-1.0
2,0.000000,5,4,46237.601562,322717.59375,1.250000,0.000000,9294.753906,145858.687500,2,4,0,0,-1.0,-1.0
3,0.122419,1356,648,50336.078125,268438.71875,2.092592,0.376112,16642.734375,124164.734375,152,158,23,120,1.5,1.6
4,0.054795,73,42,50610.929688,314545.18750,1.738095,0.229154,20720.726562,155550.312500,52,53,0,4,-1.0,1.5


In [6]:
for c in df.columns:
    if str(df[c].dtype)=='int32':
        print(c, df[c].dtype)
        df[c] = df[c].astype('float32')

count_item7 int32
count_user7 int32
prev7 int32
next7 int32
orders7 int32
carts7 int32


In [7]:
users = users.merge(df, left_on='item', right_index=True, how='left')
print( users.shape )
users.head()

(41795000, 115)


,user,item,pos,wgt,score_1,score_3,score_4,score_5,score_6,score_7,score_10,score_11,score_12,score_13,score_30,score_14,score_15,score_16,score_17,score_18,score_19,score_20,score_21,score_22,score_23,score_24,score_25,score_26,buy_ratio_5,count_item_5,count_user_5,repeat_5,buy_ratio_std_5,item_day_std_5,user_day_std_5,buy_ratio_0_6,buy_ratio_1_6,buy_ratio_2_6,buy_ratio_3_6,buy_ratio_4_6,buy_ratio_5_6,buy_ratio_6_6,count_item_0_6,count_item_1_6,count_item_2_6,count_item_3_6,count_item_4_6,count_item_5_6,count_item_6_6,count_user_0_6,count_user_1_6,count_user_2_6,count_user_3_6,count_user_4_6,count_user_5_6,count_user_6_6,item_day_std_6,user_day_std_6,buy_ratio,count_item,count_user,hour_mean,day_mean,repeat,buy_ratio_std,hour_std,day_std,buy_ratio3,count_item3,count_user3,hour_mean3,day_mean3,repeat3,buy_ratio_std3,hour_std3,day_std3,prev3,next3,orders3,carts3,order_repeat3,cart_repeat3,buy_ratio2,count_item2,unique_item2,hour_mean2,day_mean2,repeat2,buy_ratio_std2,hour_std2,day_std2,ts_max7,ts_min7,ts_diff7,z1,z2,z3,z4,z5,z6,buy_ratio7,count_item7,count_user7,hour_mean7,day_mean7,repeat7,buy_ratio_std7,hour_std7,day_std7,prev7,next7,orders7,carts7,order_repeat7,cart_repeat7
0,12899779,59625,0,-1.000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.414214,-1.0,0.071773,-1.0,-1.000,1.414214,1.414214,1.414214,1.414214,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.000000,4.0,4.0,1.000000,0.000000,NaN,NaN,0.000000,-1.00,-1.000000,-1.0,-1.0,-1.000000,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.377964,0.377964,0.000000,7.0,6.0,58189.570312,292703.84375,1.166667,0.000000,22522.156250,177373.40625,0.000000,1.0,1.0,79200.000000,338400.00000,1.000000,-1.000000,-1.000000,-1.000000,0.0,0.0,0.0,0.0,-1.0,-1.0,0.0,1.0,1.0,79200.0,338400.0,1.0,-1.0,-1.0,-1.0,0,0,0,1.000000,-1.0,-1.0,-1.0,-1.0,-1.0,0.000000,2.0,2.0,64435.000000,150835.00000,1.000000,0.000000,8051.117676,114136.937500,2.0,2.0,0.0,0.0,-1.0,-1.000000
1,12899779,737445,1,10.495,1.0,-3.0,-3.0,-3.0,2.0,-3.0,0.850000,3.0,1.500000,8.0,8.000,-3.000000,-3.000000,-3.000000,0.850000,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,1.0,2.0,1.0,0.022444,401.0,258.0,1.554264,0.148307,NaN,NaN,0.000000,0.00,0.166667,0.0,0.0,0.000000,0.0,4.0,7.0,6.0,3.0,3.0,3.0,9.0,4.0,4.0,4.0,3.0,3.0,2.0,7.0,2.380476,2.380476,0.019337,724.0,450.0,50406.179688,300655.90625,1.608889,0.137802,19708.617188,166473.00000,0.028571,35.0,25.0,50927.429688,349624.56250,1.400000,0.169031,21019.531250,161340.265625,15.0,12.0,0.0,1.0,-1.0,1.0,0.0,1.0,1.0,79200.0,338400.0,1.0,-1.0,-1.0,-1.0,0,0,0,0.954863,-1.0,-1.0,-1.0,-1.0,-1.0,0.021201,283.0,180.0,51423.164062,294747.53125,1.572222,0.144310,19882.255859,162015.390625,142.0,133.0,0.0,6.0,-1.0,1.000000
2,12899779,1253524,2,6.000,1.0,-3.0,-3.0,-3.0,0.5,-3.0,0.050000,1.5,1.500000,7.0,6.200,-3.000000,-3.000000,-3.000000,0.050000,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,1.0,0.5,2.0,0.008696,115.0,90.0,1.277778,0.093250,NaN,NaN,-1.000000,0.00,0.000000,0.0,-1.0,-1.000000,-1.0,-1.0,1.0,2.0,1.0,-1.0,-1.0,-1.0,-1.0,1.0,2.0,1.0,-1.0,-1.0,-1.0,0.786796,0.786796,0.027273,220.0,160.0,54470.640625,285001.56250,1.375000,0.163248,19156.367188,170871.09375,0.000000,4.0,4.0,53515.000000,420715.00000,1.000000,0.000000,11709.117188,255545.156250,2.0,3.0,0.0,0.0,-1.0,-1.0,0.0,1.0,1.0,79200.0,338400.0,1.0,-1.0,-1.0,-1.0,0,0,0,0.954612,-1.0,-1.0,-1.0,-1.0,-1.0,0.035294,85.0,60.0,54859.445312,283565.34375,1.416667,0.185617,17451.962891,181841.125000,47.0,43.0,0.0,3.0,-1.0,-1.000000
3,12899779,731692,3,5.990,1.0,-3.0,-3.0,-3.0,0.5,-3.0,-3.000000,1.5,0.300000,4.0,10.675,-3.000000,-3.000000,-3.000000,-3.000000,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,1.0,0.5,1.0,0.140000,50.0,28.0,1.785714,0.404566,NaN,NaN,-1.000000,0.25,0.500000,-1.0,-1.0,-1.000000,0.0,-1.0,4.0,2.0,-1.0,-1.0,-1.0,1.0,-1.0,2.0,1.0,-1.0,-1.0,-1.0,1.0,1.527525,1.527525,0.162791,86.0,41.0,52135.464844,295261.03125,2.097561,0.430061,17917.339844,174344.12500,0.285714,7.0,4.0,53310.429688,485310.43750,1.750000,0.487950,11891.376953,79692.445312,3.0,4.0,0.0,2.0,-

In [8]:
# FEAT 8
df = pd.read_parquet('../../data/item_user_features/item21_LB.pqt')
df.head()

,buy_ratio8,count_item8,count_user8,hour_mean8,day_mean8,repeat8,buy_ratio_std8,hour_std8,day_std8,prev8,next8,orders8,carts8,order_repeat8,cart_repeat8
aid,,,,,,,,,,,,,,,
0,0.000000,16,14,70702.687500,389302.68750,1.142857,0.000000,7108.441895,148486.140625,12,14,0,0,-1.0,-1.000000
1,0.066667,15,15,65013.535156,324213.53125,1.000000,0.258199,9438.640625,164628.312500,14,13,0,1,-1.0,-1.000000
2,0.000000,3,3,60675.667969,348675.65625,1.000000,0.000000,8056.046875,135972.171875,2,3,0,0,-1.0,-1.000000
3,0.116331,1341,685,62858.351562,285075.81250,1.957664,0.372419,10779.861328,150338.562500,201,196,24,108,2.4,3.117647
4,0.050000,100,62,66462.210938,300606.21875,1.612903,0.219043,12213.166992,169172.562500,69,69,0,5,-1.0,1.000000


In [9]:
for c in df.columns:
    if str(df[c].dtype)=='int32':
        print(c, df[c].dtype)
        df[c] = df[c].astype('float32')

count_item8 int32
count_user8 int32
prev8 int32
next8 int32
orders8 int32
carts8 int32


In [10]:
users = users.merge(df, left_on='item', right_index=True, how='left')
print( users.shape )
users.head()

(41795000, 130)


,user,item,pos,wgt,score_1,score_3,score_4,score_5,score_6,score_7,score_10,score_11,score_12,score_13,score_30,score_14,score_15,score_16,score_17,score_18,score_19,score_20,score_21,score_22,score_23,score_24,score_25,score_26,buy_ratio_5,count_item_5,count_user_5,repeat_5,buy_ratio_std_5,item_day_std_5,user_day_std_5,buy_ratio_0_6,buy_ratio_1_6,buy_ratio_2_6,buy_ratio_3_6,buy_ratio_4_6,buy_ratio_5_6,buy_ratio_6_6,count_item_0_6,count_item_1_6,count_item_2_6,count_item_3_6,count_item_4_6,count_item_5_6,count_item_6_6,count_user_0_6,count_user_1_6,count_user_2_6,count_user_3_6,count_user_4_6,count_user_5_6,count_user_6_6,item_day_std_6,user_day_std_6,buy_ratio,count_item,count_user,hour_mean,day_mean,repeat,buy_ratio_std,hour_std,day_std,buy_ratio3,count_item3,count_user3,hour_mean3,day_mean3,repeat3,buy_ratio_std3,hour_std3,day_std3,prev3,next3,orders3,carts3,order_repeat3,cart_repeat3,buy_ratio2,count_item2,unique_item2,hour_mean2,day_mean2,repeat2,buy_ratio_std2,hour_std2,day_std2,ts_max7,ts_min7,ts_diff7,z1,z2,z3,z4,z5,z6,buy_ratio7,count_item7,count_user7,hour_mean7,day_mean7,repeat7,buy_ratio_std7,hour_std7,day_std7,prev7,next7,orders7,carts7,order_repeat7,cart_repeat7,buy_ratio8,count_item8,count_user8,hour_mean8,day_mean8,repeat8,buy_ratio_std8,hour_std8,day_std8,prev8,next8,orders8,carts8,order_repeat8,cart_repeat8
0,12899779,59625,0,-1.000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.414214,-1.0,0.071773,-1.0,-1.000,1.414214,1.414214,1.414214,1.414214,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.000000,4.0,4.0,1.000000,0.000000,NaN,NaN,0.000000,-1.00,-1.000000,-1.0,-1.0,-1.000000,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.377964,0.377964,0.000000,7.0,6.0,58189.570312,292703.84375,1.166667,0.000000,22522.156250,177373.40625,0.000000,1.0,1.0,79200.000000,338400.00000,1.000000,-1.000000,-1.000000,-1.000000,0.0,0.0,0.0,0.0,-1.0,-1.0,0.0,1.0,1.0,79200.0,338400.0,1.0,-1.0,-1.0,-1.0,0,0,0,1.000000,-1.0,-1.0,-1.0,-1.0,-1.0,0.000000,2.0,2.0,64435.000000,150835.00000,1.000000,0.000000,8051.117676,114136.937500,2.0,2.0,0.0,0.0,-1.0,-1.000000,0.000000,6.0,5.0,66307.000000,325507.00000,1.200000,0.000000,7429.592773,169454.437500,5.0,5.0,0.0,0.0,-1.0,-1.0
1,12899779,737445,1,10.495,1.0,-3.0,-3.0,-3.0,2.0,-3.0,0.850000,3.0,1.500000,8.0,8.000,-3.000000,-3.000000,-3.000000,0.850000,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,1.0,2.0,1.0,0.022444,401.0,258.0,1.554264,0.148307,NaN,NaN,0.000000,0.00,0.166667,0.0,0.0,0.000000,0.0,4.0,7.0,6.0,3.0,3.0,3.0,9.0,4.0,4.0,4.0,3.0,3.0,2.0,7.0,2.380476,2.380476,0.019337,724.0,450.0,50406.179688,300655.90625,1.608889,0.137802,19708.617188,166473.00000,0.028571,35.0,25.0,50927.429688,349624.56250,1.400000,0.169031,21019.531250,161340.265625,15.0,12.0,0.0,1.0,-1.0,1.0,0.0,1.0,1.0,79200.0,338400.0,1.0,-1.0,-1.0,-1.0,0,0,0,0.954863,-1.0,-1.0,-1.0,-1.0,-1.0,0.021201,283.0,180.0,51423.164062,294747.53125,1.572222,0.144310,19882.255859,162015.390625,142.0,133.0,0.0,6.0,-1.0,1.000000,0.019417,412.0,255.0,60896.785156,322403.59375,1.615686,0.138155,19028.656250,166305.156250,197.0,183.0,0.0,8.0,-1.0,1.0
2,12899779,1253524,2,6.000,1.0,-3.0,-3.0,-3.0,0.5,-3.0,0.050000,1.5,1.500000,7.0,6.200,-3.000000,-3.000000,-3.000000,0.050000,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,1.0,0.5,2.0,0.008696,115.0,90.0,1.277778,0.093250,NaN,NaN,-1.000000,0.00,0.000000,0.0,-1.0,-1.000000,-1.0,-1.0,1.0,2.0,1.0,-1.0,-1.0,-1.0,-1.0,1.0,2.0,1.0,-1.0,-1.0,-1.0,0.786796,0.786796,0.027273,220.0,160.0,54470.640625,285001.56250,1.375000,0.163248,19156.367188,170871.09375,0.000000,4.0,4.0,53515.000000,420715.00000,1.000000,0.000000,11709.117188,255545.156250,2.0,3.0,0.0,0.0,-1.0,-1.0,0.0,1.0,1.0,79200.0,338400.0,1.0,-1.0,-1.0,-1.0,0,0,0,0.954612,-1.0,-1.0,-1.0,-1.0,-1.0,0.035294,85.0,60.0,54859.445312,283565.34375,1.416667,0.185617,17451.962891,181841.125000,47.0,43.0,0.0,3.0,-1.0,-1.000000,0.027972,143.0,107.0,63846.753906,280753.06250,1.336449,0.165472,16398.328125,180776.359375,72.0,74.0,0.0,4.0,-1.0,2.0
3,12899779,731692,3,5.990,1.0,-3.

In [11]:
# FEAT 9
df = pd.read_parquet('../../data/item_user_features/item22_LB.pqt')
df.head()

,buy_ratio9,count_item9,count_user9,hour_mean9,day_mean9,repeat9,buy_ratio_std9,hour_std9,day_std9,prev9,next9,orders9,carts9,order_repeat9,cart_repeat9
aid,,,,,,,,,,,,,,,
0,0.000000,17,15,32152.412109,387917.12500,1.133333,0.000000,10038.110352,152401.703125,13,15,0,0,-1.000000,-1.000000
1,0.000000,16,13,34758.625000,196758.62500,1.230769,0.000000,11851.911133,133107.281250,15,13,0,0,-1.000000,-1.000000
2,0.000000,13,12,40367.382812,279628.93750,1.083333,0.000000,6893.389160,134957.984375,9,10,0,0,-1.000000,-1.000000
3,0.088407,1199,669,36423.238281,302252.75000,1.792227,0.322538,9016.490234,144964.500000,182,171,14,78,2.333333,2.090909
4,0.047619,63,48,36173.492188,296744.90625,1.312500,0.214669,9102.734375,149799.468750,45,43,0,3,-1.000000,1.000000


In [12]:
for c in df.columns:
    if str(df[c].dtype)=='int32':
        print(c, df[c].dtype)
        df[c] = df[c].astype('float32')

count_item9 int32
count_user9 int32
prev9 int32
next9 int32
orders9 int32
carts9 int32


In [13]:
users = users.merge(df, left_on='item', right_index=True, how='left')
print( users.shape )
users.head()

(41795000, 145)


,user,item,pos,wgt,score_1,score_3,score_4,score_5,score_6,score_7,score_10,score_11,score_12,score_13,score_30,score_14,score_15,score_16,score_17,score_18,score_19,score_20,score_21,score_22,score_23,score_24,score_25,score_26,buy_ratio_5,count_item_5,count_user_5,repeat_5,buy_ratio_std_5,item_day_std_5,user_day_std_5,buy_ratio_0_6,buy_ratio_1_6,buy_ratio_2_6,buy_ratio_3_6,buy_ratio_4_6,buy_ratio_5_6,buy_ratio_6_6,count_item_0_6,count_item_1_6,count_item_2_6,count_item_3_6,count_item_4_6,count_item_5_6,count_item_6_6,count_user_0_6,count_user_1_6,count_user_2_6,count_user_3_6,count_user_4_6,count_user_5_6,count_user_6_6,item_day_std_6,user_day_std_6,buy_ratio,count_item,count_user,hour_mean,day_mean,repeat,buy_ratio_std,hour_std,day_std,buy_ratio3,count_item3,count_user3,hour_mean3,day_mean3,repeat3,buy_ratio_std3,hour_std3,day_std3,prev3,next3,orders3,carts3,order_repeat3,cart_repeat3,buy_ratio2,count_item2,unique_item2,hour_mean2,day_mean2,repeat2,buy_ratio_std2,hour_std2,day_std2,ts_max7,ts_min7,ts_diff7,z1,z2,z3,z4,z5,z6,buy_ratio7,count_item7,count_user7,hour_mean7,day_mean7,repeat7,buy_ratio_std7,hour_std7,day_std7,prev7,next7,orders7,carts7,order_repeat7,cart_repeat7,buy_ratio8,count_item8,count_user8,hour_mean8,day_mean8,repeat8,buy_ratio_std8,hour_std8,day_std8,prev8,next8,orders8,carts8,order_repeat8,cart_repeat8,buy_ratio9,count_item9,count_user9,hour_mean9,day_mean9,repeat9,buy_ratio_std9,hour_std9,day_std9,prev9,next9,orders9,carts9,order_repeat9,cart_repeat9
0,12899779,59625,0,-1.000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.414214,-1.0,0.071773,-1.0,-1.000,1.414214,1.414214,1.414214,1.414214,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.000000,4.0,4.0,1.000000,0.000000,NaN,NaN,0.000000,-1.00,-1.000000,-1.0,-1.0,-1.000000,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.377964,0.377964,0.000000,7.0,6.0,58189.570312,292703.84375,1.166667,0.000000,22522.156250,177373.40625,0.000000,1.0,1.0,79200.000000,338400.00000,1.000000,-1.000000,-1.000000,-1.000000,0.0,0.0,0.0,0.0,-1.0,-1.0,0.0,1.0,1.0,79200.0,338400.0,1.0,-1.0,-1.0,-1.0,0,0,0,1.000000,-1.0,-1.0,-1.0,-1.0,-1.0,0.000000,2.0,2.0,64435.000000,150835.00000,1.000000,0.000000,8051.117676,114136.937500,2.0,2.0,0.0,0.0,-1.0,-1.000000,0.000000,6.0,5.0,66307.000000,325507.00000,1.200000,0.000000,7429.592773,169454.437500,5.0,5.0,0.0,0.0,-1.0,-1.0,0.000000,1.0,1.0,9485.000000,95885.00000,1.000000,-1.000000,-1.000000,-1.000000,1.0,1.0,0.0,0.0,-1.0,-1.000000
1,12899779,737445,1,10.495,1.0,-3.0,-3.0,-3.0,2.0,-3.0,0.850000,3.0,1.500000,8.0,8.000,-3.000000,-3.000000,-3.000000,0.850000,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,1.0,2.0,1.0,0.022444,401.0,258.0,1.554264,0.148307,NaN,NaN,0.000000,0.00,0.166667,0.0,0.0,0.000000,0.0,4.0,7.0,6.0,3.0,3.0,3.0,9.0,4.0,4.0,4.0,3.0,3.0,2.0,7.0,2.380476,2.380476,0.019337,724.0,450.0,50406.179688,300655.90625,1.608889,0.137802,19708.617188,166473.00000,0.028571,35.0,25.0,50927.429688,349624.56250,1.400000,0.169031,21019.531250,161340.265625,15.0,12.0,0.0,1.0,-1.0,1.0,0.0,1.0,1.0,79200.0,338400.0,1.0,-1.0,-1.0,-1.0,0,0,0,0.954863,-1.0,-1.0,-1.0,-1.0,-1.0,0.021201,283.0,180.0,51423.164062,294747.53125,1.572222,0.144310,19882.255859,162015.390625,142.0,133.0,0.0,6.0,-1.0,1.000000,0.019417,412.0,255.0,60896.785156,322403.59375,1.615686,0.138155,19028.656250,166305.156250,197.0,183.0,0.0,8.0,-1.0,1.0,0.019231,312.0,215.0,36553.199219,271937.81250,1.451163,0.137556,9282.897461,162549.218750,148.0,134.0,0.0,6.0,-1.0,1.000000
2,12899779,1253524,2,6.000,1.0,-3.0,-3.0,-3.0,0.5,-3.0,0.050000,1.5,1.500000,7.0,6.200,-3.000000,-3.000000,-3.000000,0.050000,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,1.0,0.5,2.0,0.008696,115.0,90.0,1.277778,0.093250,NaN,NaN,-1.000000,0.00,0.000000,0.0,-1.0,-1.000000,-1.0,-1.0,1.0,2.0,1.0,-1.0,-1.0,-1.0,-1.0,1.0,2.0,1.0,-1.0,-1.0,-1.0,0.786796,0.786796,0.027273,220.0,160.0,54470.640625,285001.56250,1.375000,0.163248,19156.367188,170871.09375,0.000000,4.0,4.0,53515.000000,420715.00000,1.000000,0.000000,11709.117188,255545.156250,2

In [14]:
# FEAT 10
df = pd.read_parquet('../../data/item_user_features/user10_LB.pqt')
df.head()

,buy_ratio10,count_item10,unique_item10,hour_mean10,day_mean10,repeat10,buy_ratio_std10,hour_std10,day_std10
session,,,,,,,,,
12899781,1.000000,1,1,50209.000000,395809.00000,1.0,-1.000000,-1.000000,-1.000000
12899782,1.333333,24,20,65109.875000,407109.87500,1.2,0.481543,13592.392578,19700.259766
12899786,1.000000,1,1,79217.000000,338417.00000,1.0,-1.000000,-1.000000,-1.000000
12899787,1.000000,3,1,79232.000000,338432.00000,3.0,0.000000,8.888194,8.888194
12899790,1.333333,3,2,79284.335938,338484.34375,1.5,0.577350,128.165253,128.165253


In [15]:
for c in df.columns:
    if str(df[c].dtype)=='int32':
        print(c, df[c].dtype)
        df[c] = df[c].astype('float32')

count_item10 int32
unique_item10 int32


In [16]:
users = users.merge(df, left_on='user', right_index=True, how='left')
print( users.shape )
users.head()

(41795000, 154)


,user,item,pos,wgt,score_1,score_3,score_4,score_5,score_6,score_7,score_10,score_11,score_12,score_13,score_30,score_14,score_15,score_16,score_17,score_18,score_19,score_20,score_21,score_22,score_23,score_24,score_25,score_26,buy_ratio_5,count_item_5,count_user_5,repeat_5,buy_ratio_std_5,item_day_std_5,user_day_std_5,buy_ratio_0_6,buy_ratio_1_6,buy_ratio_2_6,buy_ratio_3_6,buy_ratio_4_6,buy_ratio_5_6,buy_ratio_6_6,count_item_0_6,count_item_1_6,count_item_2_6,count_item_3_6,count_item_4_6,count_item_5_6,count_item_6_6,count_user_0_6,count_user_1_6,count_user_2_6,count_user_3_6,count_user_4_6,count_user_5_6,count_user_6_6,item_day_std_6,user_day_std_6,buy_ratio,count_item,count_user,hour_mean,day_mean,repeat,buy_ratio_std,hour_std,day_std,buy_ratio3,count_item3,count_user3,hour_mean3,day_mean3,repeat3,buy_ratio_std3,hour_std3,day_std3,prev3,next3,orders3,carts3,order_repeat3,cart_repeat3,buy_ratio2,count_item2,unique_item2,hour_mean2,day_mean2,repeat2,buy_ratio_std2,hour_std2,day_std2,ts_max7,ts_min7,ts_diff7,z1,z2,z3,z4,z5,z6,buy_ratio7,count_item7,count_user7,hour_mean7,day_mean7,repeat7,buy_ratio_std7,hour_std7,day_std7,prev7,next7,orders7,carts7,order_repeat7,cart_repeat7,buy_ratio8,count_item8,count_user8,hour_mean8,day_mean8,repeat8,buy_ratio_std8,hour_std8,day_std8,prev8,next8,orders8,carts8,order_repeat8,cart_repeat8,buy_ratio9,count_item9,count_user9,hour_mean9,day_mean9,repeat9,buy_ratio_std9,hour_std9,day_std9,prev9,next9,orders9,carts9,order_repeat9,cart_repeat9,buy_ratio10,count_item10,unique_item10,hour_mean10,day_mean10,repeat10,buy_ratio_std10,hour_std10,day_std10
0,12899779,59625,0,-1.000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.414214,-1.0,0.071773,-1.0,-1.000,1.414214,1.414214,1.414214,1.414214,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.000000,4.0,4.0,1.000000,0.000000,NaN,NaN,0.000000,-1.00,-1.000000,-1.0,-1.0,-1.000000,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.377964,0.377964,0.000000,7.0,6.0,58189.570312,292703.84375,1.166667,0.000000,22522.156250,177373.40625,0.000000,1.0,1.0,79200.000000,338400.00000,1.000000,-1.000000,-1.000000,-1.000000,0.0,0.0,0.0,0.0,-1.0,-1.0,0.0,1.0,1.0,79200.0,338400.0,1.0,-1.0,-1.0,-1.0,0,0,0,1.000000,-1.0,-1.0,-1.0,-1.0,-1.0,0.000000,2.0,2.0,64435.000000,150835.00000,1.000000,0.000000,8051.117676,114136.937500,2.0,2.0,0.0,0.0,-1.0,-1.000000,0.000000,6.0,5.0,66307.000000,325507.00000,1.200000,0.000000,7429.592773,169454.437500,5.0,5.0,0.0,0.0,-1.0,-1.0,0.000000,1.0,1.0,9485.000000,95885.00000,1.000000,-1.000000,-1.000000,-1.000000,1.0,1.0,0.0,0.0,-1.0,-1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,12899779,737445,1,10.495,1.0,-3.0,-3.0,-3.0,2.0,-3.0,0.850000,3.0,1.500000,8.0,8.000,-3.000000,-3.000000,-3.000000,0.850000,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,1.0,2.0,1.0,0.022444,401.0,258.0,1.554264,0.148307,NaN,NaN,0.000000,0.00,0.166667,0.0,0.0,0.000000,0.0,4.0,7.0,6.0,3.0,3.0,3.0,9.0,4.0,4.0,4.0,3.0,3.0,2.0,7.0,2.380476,2.380476,0.019337,724.0,450.0,50406.179688,300655.90625,1.608889,0.137802,19708.617188,166473.00000,0.028571,35.0,25.0,50927.429688,349624.56250,1.400000,0.169031,21019.531250,161340.265625,15.0,12.0,0.0,1.0,-1.0,1.0,0.0,1.0,1.0,79200.0,338400.0,1.0,-1.0,-1.0,-1.0,0,0,0,0.954863,-1.0,-1.0,-1.0,-1.0,-1.0,0.021201,283.0,180.0,51423.164062,294747.53125,1.572222,0.144310,19882.255859,162015.390625,142.0,133.0,0.0,6.0,-1.0,1.000000,0.019417,412.0,255.0,60896.785156,322403.59375,1.615686,0.138155,19028.656250,166305.156250,197.0,183.0,0.0,8.0,-1.0,1.0,0.019231,312.0,215.0,36553.199219,271937.81250,1.451163,0.137556,9282.897461,162549.218750,148.0,134.0,0.0,6.0,-1.0,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,12899779,1253524,2,6.000,1.0,-3.0,-3.0,-3.0,0.5,-3.0,0.050000,1.5,1.500000,7.0,6.200,-3.000000,-3.000000,-3.000000,0.050000,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,1.0,0.5,2.0,0.008696,115.0,90.0,1.277778,0.093250,NaN,NaN,-1.000000,0.00,0.000000,0.0,-1.0,-1.000000,-1.0,-1.0,1.0,2.0,1.0,-1.0,-1.0,-1.0,-1.0,1.0,2.0,1.0,-1.0,-1.0,-1.0,0.786796,0.786796,

# Write Parquet

In [17]:
for c in users.columns:
    print(c, users[c].dtype)
    if str(users[c].dtype) in ['Float32','Int32','float64','int64']:
        print('###',c)
        users[c] = users[c].astype('float32')

user int32
item int32
pos int32
wgt float32
score_1 float32
score_3 float32
score_4 float32
score_5 float32
score_6 float32
score_7 float32
score_10 float32
score_11 float32
score_12 float32
score_13 float32
score_30 float32
score_14 float32
score_15 float32
score_16 float32
score_17 float32
score_18 float32
score_19 float32
score_20 float32
score_21 float32
score_22 float32
score_23 float32
score_24 float32
score_25 float32
score_26 float32
buy_ratio_5 float32
count_item_5 float32
count_user_5 float32
repeat_5 float32
buy_ratio_std_5 float32
item_day_std_5 float32
user_day_std_5 float32
buy_ratio_0_6 float32
buy_ratio_1_6 float32
buy_ratio_2_6 float32
buy_ratio_3_6 float32
buy_ratio_4_6 float32
buy_ratio_5_6 float32
buy_ratio_6_6 float32
count_item_0_6 float32
count_item_1_6 float32
count_item_2_6 float32
count_item_3_6 float32
count_item_4_6 float32
count_item_5_6 float32
count_item_6_6 float32
count_user_0_6 float32
count_user_1_6 float32
count_user_2_6 float32
count_user_3_6 float3

In [18]:
%%time
users.to_parquet(f'../../data/infer_with_features/train_v{VER}_LB_{PART}.pqt',index=False)

CPU times: user 3min 3s, sys: 7.64 s, total: 3min 11s
Wall time: 2min 26s
